In [1]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import re

# Sitemap URL
sitemap_url = 'https://www.essex.ac.uk/content.xml'

# Fetch and decode
response = requests.get(sitemap_url)
content = response.content.decode('utf-8', errors='replace')

# Parse XML
try:
    root = ET.fromstring(content)
    ns = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}

    # Extract URLs and lastmod
    data = []
    for url_elem in root.findall('ns:url', ns):
        loc = url_elem.find('ns:loc', ns)
        lastmod = url_elem.find('ns:lastmod', ns)
        data.append({
            'URL': loc.text if loc is not None else '',
            'Last Modified': lastmod.text if lastmod is not None else ''
        })

    df = pd.DataFrame(data)
    print(f"✅ Extracted {len(df)} URLs.")
    
    # Extract the second path segment after the domain
    df['Second Prefix'] = df['URL'].apply(lambda url: url.replace('https://www.essex.ac.uk/', '').split('/')[0])
    
    # Save the main CSV file with all data and new name
    main_filename = "essex_content_sitemap_2nd_Prefix.csv"
    df.to_csv(main_filename, index=False)
    print(f"Saved main data to '{main_filename}'")
    
    # Group by 'Second Prefix' and save each group to a CSV
    for prefix, group_df in df.groupby('Second Prefix'):
        # Sanitize prefix to create a valid filename
        safe_prefix = re.sub(r'[^a-zA-Z0-9_-]', '_', prefix)
        filename = f"essex_urls_{safe_prefix}.csv"
        group_df.to_csv(filename, index=False)
        print(f"✅ Saved {len(group_df)} URLs to {filename}")

except ET.ParseError as e:
    print("❌ XML Parse Error:", e)

✅ Extracted 8317 URLs.
Saved main data to 'essex_content_sitemap_2nd_Prefix.csv'
✅ Saved 39 URLs to essex_urls_about.csv
✅ Saved 38 URLs to essex_urls_alumni.csv
✅ Saved 2 URLs to essex_urls_apprenticeships.csv
✅ Saved 5 URLs to essex_urls_arena.csv
✅ Saved 13 URLs to essex_urls_blog.csv
✅ Saved 45 URLs to essex_urls_business.csv
✅ Saved 209 URLs to essex_urls_centres-and-institutes.csv
✅ Saved 6 URLs to essex_urls_china.csv
✅ Saved 6 URLs to essex_urls_choir.csv
✅ Saved 7 URLs to essex_urls_clearing.csv
✅ Saved 562 URLs to essex_urls_departments.csv
✅ Saved 14 URLs to essex_urls_disclaimer.csv
✅ Saved 9 URLs to essex_urls_donate.csv
✅ Saved 15 URLs to essex_urls_event-series.csv
✅ Saved 30 URLs to essex_urls_events.csv
✅ Saved 1 URLs to essex_urls_fees-and-funding.csv
✅ Saved 12 URLs to essex_urls_global.csv
✅ Saved 49 URLs to essex_urls_governance-and-strategy.csv
✅ Saved 14 URLs to essex_urls_graduation.csv
✅ Saved 108 URLs to essex_urls_international.csv
✅ Saved 16 URLs to essex_ur

In [7]:
import pandas as pd
from urllib.parse import urlparse

# Read existing CSV
df = pd.read_csv("essex_content_sitemap_2nd_Prefix.csv")

# Extract path segments from URLs
def get_hierarchy(url):
    parsed = urlparse(url)
    return [seg for seg in parsed.path.split('/') if seg]

df['Path_Segments'] = df['URL'].apply(get_hierarchy)

# Determine maximum depth needed
max_depth = df['Path_Segments'].apply(len).max()

# Create hierarchical columns
for i in range(max_depth):
    df[f'Level_{i+1}'] = df['Path_Segments'].apply(
        lambda x: x[i] if i < len(x) else pd.NA
    )

# Create final dataframe with desired columns
hierarchy_df = df[['Last Modified', 'URL'] + [f'Level_{i+1}' for i in range(max_depth)]]

# Save to new CSV
hierarchy_df.to_csv("essex_url_hierarchy_with_dates.csv", index=False)

print("✅ Hierarchical CSV created with columns:")
print(hierarchy_df.columns.tolist())
print(f"\nSample output:\n{hierarchy_df.head(3)}")

✅ Hierarchical CSV created with columns:
['Last Modified', 'URL', 'Level_1', 'Level_2', 'Level_3', 'Level_4', 'Level_5', 'Level_6', 'Level_7', 'Level_8']

Sample output:
  Last Modified                                           URL   Level_1  \
0    2022-11-07     https://www.essex.ac.uk/research/showcase  research   
1    2023-04-13         https://www.essex.ac.uk/blog/post-map      blog   
2    2023-11-10  https://www.essex.ac.uk/life/loughton-campus      life   

           Level_2 Level_3 Level_4 Level_5 Level_6 Level_7 Level_8  
0         showcase    <NA>    <NA>    <NA>    <NA>    <NA>    <NA>  
1         post-map    <NA>    <NA>    <NA>    <NA>    <NA>    <NA>  
2  loughton-campus    <NA>    <NA>    <NA>    <NA>    <NA>    <NA>  


In [5]:
import pandas as pd
# Load the CSV file (replace 'essex_url_hierarchy_with_dates.csv' with your file path)
df = pd.read_csv('essex_url_hierarchy_with_dates.csv')

# Extract unique Level_1 values and count them
unique_level1 = df['Level_1'].nunique()

# Alternatively, get the list of unique Level_1 categories
# unique_level1_list = df['Level_1'].dropna().unique()

print(f"Number of unique Level_1 categories: {unique_level1}")

Number of unique Level_1 categories: 43


In [7]:
import pandas as pd
import requests
from requests.exceptions import RequestException
import time

# Load the CSV and get unique Level_1 categories
df = pd.read_csv('essex_url_hierarchy_with_dates.csv')
unique_level1 = df['Level_1'].dropna().unique().tolist()

base_url = "https://www.essex.ac.uk/"
results = {}

# Configure headers to mimic a real browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

for category in unique_level1:
    url = f"{base_url}{category}"
    
    try:
        # Use HEAD first to check existence without downloading content
        response = requests.head(url, headers=headers, allow_redirects=True, timeout=10)
        status = response.status_code
        
        # Consider 2xx and 3xx status codes as valid
        if 200 <= status < 400:
            results[url] = {
                'exists': True,
                'status_code': status,
                'final_url': response.url  # Show final URL after redirects
            }
        else:
            results[url] = {
                'exists': False,
                'status_code': status,
                'final_url': url
            }
            
    except RequestException as e:
        results[url] = {
            'exists': False,
            'status_code': str(e),
            'final_url': url
        }
    
    # Be polite - add delay between requests
    time.sleep(1)

# Print results
print(f"{'URL':<50} | {'Exists':<6} | {'Status':<10} | {'Final URL'}")
print("-" * 90)
for url, data in results.items():
    print(f"{url:<50} | {str(data['exists']):<6} | {str(data['status_code']):<10} | {data['final_url']}")

# Optional: Save to CSV
pd.DataFrame.from_dict(results, orient='index').to_csv('url_validation_results.csv')

URL                                                | Exists | Status     | Final URL
------------------------------------------------------------------------------------------
https://www.essex.ac.uk/research                   | True   | 200        | https://www.essex.ac.uk/research
https://www.essex.ac.uk/blog                       | True   | 200        | https://www.essex.ac.uk/blog
https://www.essex.ac.uk/life                       | True   | 200        | https://www.essex.ac.uk/life
https://www.essex.ac.uk/student                    | True   | 200        | https://www.essex.ac.uk/student
https://www.essex.ac.uk/staff                      | True   | 200        | https://www.essex.ac.uk/staff
https://www.essex.ac.uk/centres-and-institutes     | True   | 200        | https://www.essex.ac.uk/centres-and-institutes
https://www.essex.ac.uk/sport                      | True   | 200        | https://www.essex.ac.uk/sport
https://www.essex.ac.uk/about                      | True   | 200    

In [11]:
import pandas as pd
import requests
from requests.exceptions import RequestException
import time

def check_url(url):
    """Check if a URL exists with fallback from HEAD to GET requests"""
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        # Try HEAD request first
        response = requests.head(url, headers=headers, allow_redirects=True, timeout=10)
        
        # If HEAD not allowed, fallback to GET
        if response.status_code == 405:
            response = requests.get(url, headers=headers, allow_redirects=True, timeout=10)
            
        return response
    
    except RequestException as e:
        return None

def validate_level1_urls(csv_path, output_file='validation_results.csv'):
    """Main validation function"""
    # Load CSV data
    df = pd.read_csv(csv_path)
    
    # Get unique Level_1 categories
    unique_level1 = df['Level_1'].dropna().unique().tolist()
    
    base_url = "https://www.essex.ac.uk/"
    results = []

    print(f"Checking {len(unique_level1)} URLs...\n")
    
    for idx, category in enumerate(unique_level1, 1):
        url = f"{base_url}{category.strip().lower()}"
        result = {'url': url, 'exists': False, 'status_code': None, 'final_url': None}
        
        # Get HTTP response
        response = check_url(url)
        
        if response:
            result.update({
                'exists': 200 <= response.status_code < 400,
                'status_code': response.status_code,
                'final_url': response.url
            })
        else:
            result['status_code'] = 'Connection Failed'
        
        results.append(result)
        
        # Progress update
        print(f"Checked {idx}/{len(unique_level1)}: {url} => {result['status_code']}")
        
        # Be polite - add delay
        time.sleep(0.5)
    
    # Save results
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_file, index=False)
    print(f"\nResults saved to {output_file}")
    
    return results_df

if __name__ == "__main__":
    # Run validation
    results = validate_level1_urls(
        csv_path='essex_url_hierarchy_with_dates.csv',
        output_file='url_validation_report.csv'
    )

    # Print summary
    print("\nValidation Summary:")
    print(f"Valid URLs: {results['exists'].sum()}/{len(results)}")
    print(f"Invalid URLs: {len(results) - results['exists'].sum()}")
    print("\nDetailed Status Codes:")
    print(results['status_code'].value_counts())

Checking 43 URLs...

Checked 1/43: https://www.essex.ac.uk/research => 200
Checked 2/43: https://www.essex.ac.uk/blog => 200
Checked 3/43: https://www.essex.ac.uk/life => 200
Checked 4/43: https://www.essex.ac.uk/student => 200
Checked 5/43: https://www.essex.ac.uk/staff => 200
Checked 6/43: https://www.essex.ac.uk/centres-and-institutes => 200
Checked 7/43: https://www.essex.ac.uk/sport => 200
Checked 8/43: https://www.essex.ac.uk/about => 200
Checked 9/43: https://www.essex.ac.uk/test => Connection Failed
Checked 10/43: https://www.essex.ac.uk/departments => 200
Checked 11/43: https://www.essex.ac.uk/governance-and-strategy => 200
Checked 12/43: https://www.essex.ac.uk/postgraduate => 200
Checked 13/43: https://www.essex.ac.uk/wivenhoe-park => 200
Checked 14/43: https://www.essex.ac.uk/event-series => 200
Checked 15/43: https://www.essex.ac.uk/jobs => 200
Checked 16/43: https://www.essex.ac.uk/global => 200
Checked 17/43: https://www.essex.ac.uk/business => 200
Checked 18/43: https:/

In [13]:
import pandas as pd
import requests
from requests.exceptions import RequestException
import time

def check_url(url):
    """Verify URL exists with browser-like headers"""
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.head(url, headers=headers, 
                               allow_redirects=True, timeout=10)
        return response.status_code == 200
    except RequestException:
        return False

# 1. Load original data
df = pd.read_csv("essex_url_hierarchy_with_dates.csv")

# 2. Get unique Level_1 categories
base_url = "https://www.essex.ac.uk/"
level1_categories = df['Level_1'].dropna().unique()

# 3. Find missing valid URLs
new_urls = []
for category in level1_categories:
    url = f"{base_url}{category.strip().lower()}"
    
    # Skip existing URLs
    if url in df['URL'].values:
        continue
        
    # Validate new URL
    if check_url(url):
        new_urls.append({
            'URL': url,
            'Level_1': category,
            'Last Modified': pd.Timestamp.now().strftime('%Y-%m-%d')
        })
    time.sleep(0.5)  # Rate limiting

# 4. Add new URLs and clean data
if new_urls:
    new_df = pd.DataFrame(new_urls)
    df = pd.concat([df, new_df], ignore_index=True)

# 5. Remove duplicates (keep first occurrence)
df = df.drop_duplicates(subset=['URL'], keep='first')

# 6. Save updated file (overwrite original)
df.to_csv("essex_url_hierarchy_with_dates.csv", index=False)

print(f"""
Updated CSV report:
- Original entries: {len(df) - len(new_urls)}
- New valid URLs added: {len(new_urls)}
- Final total entries: {len(df)}
- Duplicates removed: {(len(df) - len(new_urls)) - (len(df) - df.duplicated(subset=['URL']).sum())}
""")


Updated CSV report:
- Original entries: 8315
- New valid URLs added: 3
- Final total entries: 8318
- Duplicates removed: -3



In [16]:
import pandas as pd

# Read source file with 'Subject URL' column
source_df = pd.read_csv('essex_subject_links_selenium.csv')
source_df.rename(columns={'Subject URL': 'URL'}, inplace=True)

# Read target file (create empty DataFrame if it doesn't exist)
try:
    target_df = pd.read_csv('essex_url_hierarchy_with_dates.csv')
except FileNotFoundError:
    target_df = pd.DataFrame(columns=['URL'])  # Assumes target has at least 'URL' column

# Combine and deduplicate
combined_df = pd.concat([target_df, source_df[['URL']]], ignore_index=True)
combined_df = combined_df.drop_duplicates(subset=['URL'], keep='first')

# Save updated file
combined_df.to_csv('essex_url_hierarchy_with_dates.csv', index=False)

print(f"Added {len(source_df)} URLs. Total unique URLs: {len(combined_df)}")

Added 60 URLs. Total unique URLs: 8378


In [18]:
import pandas as pd

def merge_urls():
    # Read subject URLs
    subjects_df = pd.read_csv('essex_subject_links_selenium.csv')
    subjects_df.rename(columns={'Subject URL': 'URL'}, inplace=True)
    subjects_urls = subjects_df[['URL']]

    # Read course URLs
    courses_df = pd.read_csv('essex_courses_simplified.csv')
    courses_urls = courses_df[['course_url']].rename(columns={'course_url': 'URL'})

    # Combine all new URLs
    new_urls = pd.concat([subjects_urls, courses_urls], ignore_index=True)

    # Read target file (create if missing)
    try:
        target_df = pd.read_csv('essex_url_hierarchy_with_dates.csv')
    except FileNotFoundError:
        target_df = pd.DataFrame(columns=['URL'])

    # Merge and deduplicate
    combined_df = pd.concat([target_df, new_urls], ignore_index=True)
    combined_df = combined_df.drop_duplicates(subset=['URL'], keep='first')

    # Save results
    combined_df.to_csv('essex_url_hierarchy_with_dates.csv', index=False)
    print(f"Added {len(new_urls)} URLs. Total unique URLs: {len(combined_df)}")

if __name__ == "__main__":
    merge_urls()

Added 745 URLs. Total unique URLs: 9063


In [20]:
import pandas as pd
from urllib.parse import urlparse

def extract_url_hierarchy(url):
    """Extract hierarchical components from Essex URLs"""
    parsed = urlparse(url)
    path_parts = [p for p in parsed.path.split('/') if p]  # Split and remove empty strings
    
    hierarchy = {
        'level_1': None,
        'level_2': None,
        'level_3': None,
        'level_4': None,
        'level_5': None
    }
    
    # Assign parts to hierarchy levels starting from first path component
    for i, part in enumerate(path_parts[:5], start=1):
        hierarchy[f'level_{i}'] = part
            
    return hierarchy

def update_hierarchy_columns(df):
    """Update DataFrame with proper hierarchy levels"""
    # Extract hierarchy components
    hierarchies = df['URL'].apply(extract_url_hierarchy).apply(pd.Series)
    
    # Merge with original dataframe
    updated_df = pd.concat([df, hierarchies], axis=1)
    
    return updated_df

# Load CSV file
df = pd.read_csv('essex_url_hierarchy_with_dates.csv')

# Update hierarchy columns
updated_df = update_hierarchy_columns(df)

# Save updated file
updated_df.to_csv('essex_url_hierarchy_with_dates.csv', index=False)

print("URL hierarchy levels updated successfully!")
print(f"Sample entry:\n{updated_df.iloc[0]}")

URL hierarchy levels updated successfully!
Sample entry:
Last Modified                                   2022-11-07
URL              https://www.essex.ac.uk/research/showcase
Level_1                                           research
Level_2                                           showcase
Level_3                                                NaN
Level_4                                                NaN
Level_5                                                NaN
Level_6                                                NaN
Level_7                                                NaN
Level_8                                                NaN
level_1                                           research
level_2                                           showcase
level_3                                               None
level_4                                               None
level_5                                               None
Name: 0, dtype: object


In [23]:
import pandas as pd
from urllib.parse import urlparse

def process_url_hierarchy(df):
    """Dynamically create hierarchy columns based on URL structure"""
    # Remove existing level columns if they exist
    level_cols = [col for col in df.columns if col.startswith('level_')]
    df = df.drop(columns=level_cols, errors='ignore')
    
    # Extract path components from URLs
    df['parts'] = df['URL'].apply(
        lambda x: [p for p in urlparse(x).path.split('/') if p]
    )
    
    # Determine maximum depth across all URLs
    max_depth = df['parts'].apply(len).max()
    
    # Create dynamic level columns
    for i in range(max_depth):
        df[f'level_{i+1}'] = df['parts'].apply(
            lambda x: x[i] if i < len(x) else None
        )
    
    # Clean up temporary column
    df = df.drop(columns=['parts'])
    
    return df

# Load the CSV file
df = pd.read_csv('essex_url_hierarchy_with_dates.csv')

# Process URL hierarchy
processed_df = process_url_hierarchy(df)

# Save updated file
processed_df.to_csv('essex_url_hierarchy_with_dates.csv', index=False)

print("Dynamic hierarchy columns created successfully!")
print(f"Created levels: {[col for col in processed_df.columns if col.startswith('level_')]}")
print("\nSample entry:")
print(processed_df.iloc[0])

Dynamic hierarchy columns created successfully!
Created levels: ['level_1', 'level_2', 'level_3', 'level_4', 'level_5', 'level_6', 'level_7', 'level_8']

Sample entry:
Last Modified                                   2022-11-07
URL              https://www.essex.ac.uk/research/showcase
Level_1                                           research
Level_2                                           showcase
Level_3                                                NaN
Level_4                                                NaN
Level_5                                                NaN
Level_6                                                NaN
Level_7                                                NaN
Level_8                                                NaN
level_1                                           research
level_2                                           showcase
level_3                                               None
level_4                                               None
level_

In [26]:
import pandas as pd

def clean_duplicate_columns(df):
    """Remove duplicate columns with case variations and identical data"""
    # Standardize column names to lowercase
    df.columns = df.columns.str.lower()
    
    # Track columns to keep (first occurrence of each duplicate group)
    columns_to_keep = []
    seen_columns = set()
    
    # Check columns in reverse order to keep first occurrence
    for col in reversed(df.columns):
        base_name = col.split('.')[0]  # Handle pandas suffixes
        if base_name not in seen_columns:
            seen_columns.add(base_name)
            columns_to_keep.append(col)
    
    # Reverse back to original order and select columns
    return df[columns_to_keep[::-1]]

def verify_cleanup(df):
    """Check for remaining duplicates and show sample structure"""
    print("\nCleanup verification:")
    print(f"Current columns: {list(df.columns)}")
    print("\nSample hierarchy structure:")
    sample = df[df['url'].str.contains('/courses/')].iloc[0]
    print(f"URL: {sample['url']}")
    for col in [c for c in df.columns if c.startswith('level_')]:
        print(f"{col}: {sample[col]}")

# Load CSV file
df = pd.read_csv('essex_url_hierarchy_with_dates.csv')

# Clean duplicate columns
cleaned_df = clean_duplicate_columns(df)

# Save cleaned file
cleaned_df.to_csv('essex_url_hierarchy_with_dates_cleaned.csv', index=False)

# Verify results
print("Column cleanup completed successfully!")
verify_cleanup(cleaned_df)

Column cleanup completed successfully!

Cleanup verification:
Current columns: ['last modified', 'url', 'level_1', 'level_1', 'level_2', 'level_2', 'level_3', 'level_3', 'level_4', 'level_4', 'level_5', 'level_5', 'level_6', 'level_6', 'level_7', 'level_7', 'level_8', 'level_8']

Sample hierarchy structure:
URL: https://www.essex.ac.uk/courses/UG00001/1/BSc-Accounting
level_1: level_1        NaN
level_1    courses
Name: 8378, dtype: object
level_1: level_1        NaN
level_1    courses
Name: 8378, dtype: object
level_2: level_2        NaN
level_2    UG00001
Name: 8378, dtype: object
level_2: level_2        NaN
level_2    UG00001
Name: 8378, dtype: object
level_3: level_3    NaN
level_3      1
Name: 8378, dtype: object
level_3: level_3    NaN
level_3      1
Name: 8378, dtype: object
level_4: level_4               NaN
level_4    BSc-Accounting
Name: 8378, dtype: object
level_4: level_4               NaN
level_4    BSc-Accounting
Name: 8378, dtype: object
level_5: level_5    NaN
level_5  

In [28]:
import pandas as pd

def clean_columns(input_file, output_file):
    """Keep only 'url' and 'last modified' columns"""
    # Read CSV file
    df = pd.read_csv(input_file)
    
    # Standardize column names (case-insensitive and space handling)
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    
    # Identify columns to keep
    keep_columns = []
    if 'url' in df.columns:
        keep_columns.append('url')
    if 'last_modified' in df.columns:
        keep_columns.append('last_modified')
    
    # Filter columns
    if keep_columns:
        cleaned_df = df[keep_columns]
    else:
        raise ValueError("No valid columns found to keep")
    
    # Save cleaned file
    cleaned_df.to_csv(output_file, index=False)
    return cleaned_df

if __name__ == "__main__":
    input_csv = "essex_url_hierarchy_with_dates.csv"
    output_csv = "essex_url_cleaned.csv"
    
    result = clean_columns(input_csv, output_csv)
    print(f"Cleaning complete! Kept columns: {list(result.columns)}")
    print(f"Total entries preserved: {len(result)}")
    print(f"Saved to: {output_csv}")

Cleaning complete! Kept columns: ['url', 'last_modified']
Total entries preserved: 9063
Saved to: essex_url_cleaned.csv


In [30]:
import pandas as pd
from urllib.parse import urlparse

def process_hierarchy(input_file, output_file):
    # Read CSV with proper column handling
    df = pd.read_csv(input_file)
    
    # Standardize column names
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    
    # Ensure required columns exist
    if 'url' not in df.columns:
        raise ValueError("URL column missing from dataset")
    
    # Create temporary parts column
    df['parts'] = df['url'].apply(
        lambda x: [p for p in urlparse(x).path.split('/') if p]
    )
    
    # Find maximum URL depth
    max_depth = df['parts'].apply(len).max()
    
    # Create dynamic level columns
    for i in range(max_depth):
        df[f'level_{i+1}'] = df['parts'].apply(
            lambda x: x[i] if i < len(x) else None
        )
    
    # Clean up temporary column
    df = df.drop(columns=['parts'])
    
    # Reorder columns with last_modified first if it exists
    column_order = []
    if 'last_modified' in df.columns:
        column_order.append('last_modified')
    column_order += ['url'] + [f'level_{i+1}' for i in range(max_depth)]
    
    # Final dataframe with sorted columns
    final_df = df[column_order]
    
    # Save to new file
    final_df.to_csv(output_file, index=False)
    return final_df

if __name__ == "__main__":
    input_csv = "essex_url_cleaned.csv"
    output_csv = "essex_url_hierarchy_final.csv"
    
    result = process_hierarchy(input_csv, output_csv)
    
    print("Processing complete!")
    print(f"Result columns: {list(result.columns)}")
    print(f"Sample entry:\n{result.iloc[0]}")
    print(f"\nSaved to: {output_csv}")

Processing complete!
Result columns: ['last_modified', 'url', 'level_1', 'level_2', 'level_3', 'level_4', 'level_5', 'level_6', 'level_7', 'level_8']
Sample entry:
last_modified                                   2022-11-07
url              https://www.essex.ac.uk/research/showcase
level_1                                           research
level_2                                           showcase
level_3                                               None
level_4                                               None
level_5                                               None
level_6                                               None
level_7                                               None
level_8                                               None
Name: 0, dtype: object

Saved to: essex_url_hierarchy_final.csv


In [32]:
import pandas as pd

def analyze_urls(file_path):
    # Read CSV file
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found")
        return
    except Exception as e:
        print(f"Error reading file: {str(e)}")
        return

    # Check for URL column
    url_column = None
    for col in df.columns:
        if col.lower() in ['url', 'urls', 'link']:
            url_column = col
            break
            
    if not url_column:
        print("No URL column found in the file")
        return
        
    # Get URL statistics
    total_urls = len(df)
    unique_urls = df[url_column].nunique()
    duplicate_count = df.duplicated(subset=[url_column]).sum()
    missing_urls = df[url_column].isnull().sum()
    
    # Print report
    print(f"URL Analysis Report for: {file_path}")
    print("="*50)
    print(f"Total URLs: {total_urls}")
    print(f"Unique URLs: {unique_urls}")
    print(f"Duplicate URLs: {duplicate_count}")
    print(f"Missing/Empty URLs: {missing_urls}")
    print("\nSample Duplicate URLs:")
    print(df[df.duplicated(subset=[url_column], keep=False)][url_column].head(5).to_string(index=False))
    
    # Save results to text file
    with open("url_analysis_report.txt", "w") as f:
        f.write(f"URL Analysis Report\n")
        f.write(f"File: {file_path}\n")
        f.write(f"Total URLs: {total_urls}\n")
        f.write(f"Unique URLs: {unique_urls}\n")
        f.write(f"Duplicate URLs: {duplicate_count}\n")
        f.write(f"Missing/Empty URLs: {missing_urls}\n")
    
    print("\nReport saved to: url_analysis_report.txt")

if __name__ == "__main__":
    file_path = "essex_url_hierarchy_final.csv"  # Update with your filename
    analyze_urls(file_path)

URL Analysis Report for: essex_url_hierarchy_final.csv
Total URLs: 9063
Unique URLs: 9063
Duplicate URLs: 0
Missing/Empty URLs: 0

Sample Duplicate URLs:
Series([], )

Report saved to: url_analysis_report.txt


In [34]:
import pandas as pd
from urllib.parse import urlparse
from datetime import datetime

def add_url_to_hierarchy(file_path, new_url):
    # Read existing CSV
    df = pd.read_csv(file_path)
    
    # Check if URL already exists
    if new_url in df['url'].values:
        print("URL already exists in file")
        return df
    
    # Create new entry
    new_entry = {
        'last_modified': datetime.today().strftime('%Y-%m-%d'),
        'url': new_url
    }
    
    # Parse URL components
    parsed = urlparse(new_url)
    path_parts = [p for p in parsed.path.split('/') if p]
    
    # Add level columns
    for i, part in enumerate(path_parts, 1):
        new_entry[f'level_{i}'] = part
    
    # Create DataFrame from new entry
    new_row = pd.DataFrame([new_entry])
    
    # Concatenate with existing data
    updated_df = pd.concat([df, new_row], ignore_index=True)
    
    # Reorder columns to match original structure
    cols = ['last_modified', 'url'] + [c for c in updated_df.columns if c.startswith('level_')]
    updated_df = updated_df[cols]
    
    # Save updated file
    updated_df.to_csv(file_path, index=False)
    return updated_df

# Configuration
file_path = "essex_url_hierarchy_final.csv"
new_url = "https://www.essex.ac.uk/apprenticeships/health-and-social-care-apprenticeships"

# Add the new URL
updated_df = add_url_to_hierarchy(file_path, new_url)

# Show confirmation
print("URL added successfully!")
print("\nNew entry structure:")
print(updated_df[['last_modified', 'url', 'level_1', 'level_2']].tail(1))

URL added successfully!

New entry structure:
     last_modified                                                url  \
9063    2025-05-17  https://www.essex.ac.uk/apprenticeships/health...   

              level_1                                 level_2  
9063  apprenticeships  health-and-social-care-apprenticeships  
